In [1]:
pip install pycryptodome

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [1]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes

# Generate RSA keys
key_alice = RSA.generate(2048)
key_bob = RSA.generate(2048)

alice_private = key_alice
alice_public = key_alice.publickey()

bob_private = key_bob
bob_public = key_bob.publickey()

# Message
message = b'Hello Bob! This is Alice.'

# Step 1: Encrypt the message with AES
session_key = get_random_bytes(16)
cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(message)

# Step 2: Encrypt session key with Bob's public RSA key
cipher_rsa = PKCS1_OAEP.new(bob_public)
enc_session_key = cipher_rsa.encrypt(session_key)

# Step 3: Alice signs the message
h = SHA256.new(message)
signature = pkcs1_15.new(alice_private).sign(h)

# ---- Send to Bob: enc_session_key, ciphertext, nonce, tag, signature ----

# Step 4: Bob decrypts session key
cipher_rsa_dec = PKCS1_OAEP.new(bob_private)
session_key_dec = cipher_rsa_dec.decrypt(enc_session_key)

# Step 5: Bob decrypts message
cipher_aes_dec = AES.new(session_key_dec, AES.MODE_EAX, nonce=cipher_aes.nonce)
message_dec = cipher_aes_dec.decrypt_and_verify(ciphertext, tag)

# Step 6: Bob verifies signature
h2 = SHA256.new(message_dec)
try:
    pkcs1_15.new(alice_public).verify(h2, signature)
    print("Signature is valid.")
except (ValueError, TypeError):
    print("Signature is invalid.")

# Output
print("Decrypted message:", message_dec.decode())


TypeError: argument 2 must be bytes, not bytearray

In [2]:
import Crypto
print(Crypto.__version__)

3.23.0


In [3]:
pip install --upgrade pycryptodome

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_EAX)
plaintext = b"Hello world!"
ct, tag = cipher.encrypt_and_digest(plaintext)
print(ct, tag)

TypeError: argument 2 must be bytes, not bytearray

In [4]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature
import os

# --- RSA Key Generation ---
backend = default_backend()

alice_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=backend
)
alice_public_key = alice_private_key.public_key()

bob_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=backend
)
bob_public_key = bob_private_key.public_key()

# --- Message ---
message = b"Hello Bob! This is Alice."

# --- Step 1: Symmetric Encryption with AES-GCM ---
aes_key = os.urandom(32)  # AES-256
nonce = os.urandom(12)    # 12 bytes recommended for GCM

encryptor = Cipher(
    algorithms.AES(aes_key),
    modes.GCM(nonce),
    backend=backend
).encryptor()

ciphertext = encryptor.update(message) + encryptor.finalize()
tag = encryptor.tag

# --- Step 2: Encrypt AES key with Bob’s RSA public key ---
enc_aes_key = bob_public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# --- Step 3: Alice signs the message with her private RSA key ---
signature = alice_private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# --------------------------
# --- Transmission Step ---
# (enc_aes_key, nonce, tag, ciphertext, signature) sent to Bob
# --------------------------

# --- Step 4: Bob decrypts the AES key ---
decrypted_aes_key = bob_private_key.decrypt(
    enc_aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# --- Step 5: Bob decrypts the ciphertext ---
decryptor = Cipher(
    algorithms.AES(decrypted_aes_key),
    modes.GCM(nonce, tag),
    backend=backend
).decryptor()

decrypted_message = decryptor.update(ciphertext) + decryptor.finalize()

# --- Step 6: Bob verifies the signature ---
try:
    alice_public_key.verify(
        signature,
        decrypted_message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("Signature is valid.")
except InvalidSignature:
    print("Signature is invalid.")

print("Decrypted message:", decrypted_message.decode())



Signature is valid.
Decrypted message: Hello Bob! This is Alice.


In [6]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature
import os

# === Generate RSA keys for Alice and Bob ===
backend = default_backend()

alice_private = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=backend
)
alice_public = alice_private.public_key()

bob_private = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=backend
)
bob_public = bob_private.public_key()

# === Message to be sent ===
message = b'Hello Bob! This is Alice.'

# === Step 1: Encrypt message with AES (EAX → GCM equivalent) ===
session_key = os.urandom(16)     # AES-128
nonce = os.urandom(12)           # GCM requires 12-byte nonce

encryptor = Cipher(
    algorithms.AES(session_key),
    modes.GCM(nonce),
    backend=backend
).encryptor()

ciphertext = encryptor.update(message) + encryptor.finalize()
tag = encryptor.tag

# === Step 2: Encrypt session key with Bob's public RSA key (OAEP) ===
enc_session_key = bob_public.encrypt(
    session_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# === Step 3: Alice signs the original message (RSA + SHA256 + PSS) ===
signature = alice_private.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# -------------------------------
# Simulate sending:
# enc_session_key, nonce, tag, ciphertext, signature
# -------------------------------

# === Step 4: Bob decrypts session key with his private key ===
session_key_dec = bob_private.decrypt(
    enc_session_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# === Step 5: Bob decrypts the message ===
decryptor = Cipher(
    algorithms.AES(session_key_dec),
    modes.GCM(nonce, tag),
    backend=backend
).decryptor()

message_dec = decryptor.update(ciphertext) + decryptor.finalize()

# === Step 6: Bob verifies Alice's signature ===
try:
    alice_public.verify(
        signature,
        message_dec,
        padding.PSS(
            mgf=padding.MGF1(h


SyntaxError: unexpected EOF while parsing (<ipython-input-6-41d4ab216e95>, line 91)